# Converting a latent space vector to an image quickly
## [Converting-latents-to-rgb-without-encoding](https://discuss.huggingface.co/t/decoding-latents-to-rgb-without-upscaling/23204)

This notebooks demonstrated a script in ./ga/latent_to_image.py, which as it name implies, takes a latent space vector (gathered from denoising an embedding using stable diffusion), and converts it to a usable PIL image.

**The script also provides a method to decode the latent by loading and using Stable Diffusion's autoencoder, though it is not demonstrated on this notebook.**

In [ ]:
ENV_TYPE = "TES1T"

if(ENV_TYPE != "TEST"):
  !git clone "https://github.com/kk-digital/kcg-ml-sd1p4.git"
  %cd kcg-ml-sd1p4
  !pip3 install -r requirements.txt
  exit()
  base_directory = "./"
else:
  base_directory = "../"

# Magical check for fixing all of our directory issues
import subprocess
output = subprocess.check_output(["pwd"], universal_newlines=True)
if "notebooks" in output:
    %cd ..
del output

In [1]:
# Check for dependency needed for using OpenCV
import subprocess

result = subprocess.run(['dpkg', '-s', 'libgl1-mesa-glx'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# If the package is not installed, install it
if 'is not installed and no information is available' in result.stderr:
    print("Installing libgl, which is needed to run the GA script.")
    subprocess.run(["apt", "update"])
    subprocess.run(["apt", "install", "libgl1-mesa-glx"])
else:
    print("Package 'libgl1-mesa-glx' is already installed.")

In [ ]:
!python3 ./download_models.py

In [ ]:
!python3 ./process_models.py

In [ ]:
import os
import sys
import torch
import time
import shutil
from torchvision.transforms import ToPILImage
from os.path import join

base_directory = "../"
sys.path.insert(0, base_directory)

from stable_diffusion.model_paths import *
from configs.model_config import ModelPathConfig
from stable_diffusion.utils_backend import *
from stable_diffusion.utils_image import *
from stable_diffusion.utils_model import *
from stable_diffusion.stable_diffusion import StableDiffusion
from utility.labml import monit


output_base_dir = join(base_directory, "./output/sd2-notebook/")
output_directory = join(output_base_dir, "latent_to_image/")


try:
    shutil.rmtree(output_directory)
except Exception as e:
    print(e, "\n", "Creating the path...")
    os.makedirs(output_directory, exist_ok=True)
else:
    os.makedirs(output_directory, exist_ok=True)


def to_pil(image):
    return ToPILImage()(torch.clamp((image + 1.0) / 2.0, min=0.0, max=1.0))

In [8]:
device = get_device()
base_dir = os.getcwd()
sys.path.insert(0, base_dir)

batch_size = 1
model_config = ModelPathConfig()
pt = IODirectoryTree(model_config)

In [ ]:
# initialize an empty stable diffusion class
stable_diffusion = StableDiffusion(device=device)
get_memory_status(device)

In [ ]:
# initialize an empty latent diffusion model; it returns self.model
# then load the clip text embedder from the path `pt.embedder_path` with .load_clip_embedder()
# it returns the clip embedder, so you can chain a .load_submodels() to load the text embedder submodels

stable_diffusion.quick_initialize().load_clip_embedder().load_submodels()
get_memory_status(device)

In [ ]:
stable_diffusion.model.clip_embedder

In [12]:
# get the embedding for a prompt
prompt_embedding = stable_diffusion.model.clip_embedder(
    ["A woman with flowers in her hair in a courtyard, in the style of Frank Frazetta"]
)
null_prompt = stable_diffusion.model.clip_embedder([""])

In [ ]:
get_memory_status(device)
prompt_embedding.shape, null_prompt.shape

In [ ]:
# we don't need the embedder anymore, so we can unload it
stable_diffusion.model.unload_clip_embedder()
get_memory_status(device)

In [15]:
# let's save the prompt embedding
torch.save(prompt_embedding, join(output_directory, "prompt_embedding.pt"))
torch.save(null_prompt, join(output_directory, "null_prompt.pt"))

In [16]:
%%capture
stable_diffusion.model.load_unet()

In [ ]:
latent_space_vector = stable_diffusion.generate_images_latent_from_embeddings(
    embedded_prompt=prompt_embedding, null_prompt=null_prompt
)

In [ ]:
from ga.latent_to_image import latent_to_pil_image

# Example usage of the function with the 'images' variable
pil_image = latent_to_pil_image(latent_space_vector, size=512)
pil_image

In [ ]:
from ga.latent_to_image import generate_image_from_latent

final = generate_image_from_latent(stable_diffusion, latent_space_vector)
final